In [56]:
#In case reader wants to input arguments ar commandline
#db = 'imdb' 
#parser = argparse.ArgumentParser(description='Gender and Age Detection')
#parser.add_argument('-i', '--input', required='True', type=str, help='Input dir for images')
#args = parser.parse_args("--i data/{}_crop/{}.mat".format(db, db).split()) #set the values of the passed variables

#metadata_path=args.input


In [57]:
import os
import sys
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.io import loadmat

dbs=['imdb','wiki']

for i, db in enumerate(dbs):
    parent_dir='data/{}/'.format(db)
    metadata_path=parent_dir+'{}.mat'.format(db)
    db_mat = loadmat(metadata_path) #https://towardsdatascience.com/how-to-load-matlab-mat-files-in-python-1f200e1287b5
    db_data=db_mat[db] #https://docs.scipy.org/doc/scipy/reference/generated/scipy.io.loadmat.html
#    print(db,db_mat.keys())
#    print(i,db,db_data.dtype)
    
    if(db=='imdb'):
        dob=db_data[0,0]['dob'][0]
        photo_taken=db_data[0,0]['photo_taken'][0]
        full_path=db_data[0,0]['full_path'][0]
        gender=db_data[0,0]['gender'][0]
        face_score=db_data[0,0]['face_score'][0]
        second_face_score=db_data[0,0]['second_face_score'][0]
        imdb_df=pd.DataFrame({"dob":dob,"photo_taken":photo_taken,"full_path": full_path,"gender": gender,"face_score": face_score,"second_face_score":second_face_score})        
        no_of_images=len(full_path)
        print(db,no_of_images)
    else:
        dob=db_data[0,0]['dob'][0]
        photo_taken=db_data[0,0]['photo_taken'][0]
        full_path=db_data[0,0]['full_path'][0]
        gender=db_data[0,0]['gender'][0]
        face_score=db_data[0,0]['face_score'][0]
        second_face_score=db_data[0,0]['second_face_score'][0]
        wiki_df=pd.DataFrame({"dob":dob,"photo_taken":photo_taken,"full_path": full_path,"gender": gender,"face_score": face_score,"second_face_score":second_face_score})
        no_of_images=len(full_path)
        print(db,no_of_images)        
        
complete_df = pd.concat([imdb_df,wiki_df],axis=0)    

imdb 460723
wiki 62328


In [58]:
# Calculate age at the time the photo was taken: get difference between 'dob' and 'photo_taken'

# 'dob' is given as a Matlab serial date number. In MATLAB, serial date number 1 is January 1, 0000. Serial time can 
# represent fractions of days beginning at midnight; for example, 6 p.m. equals 0.75 serial days. We need to utilize
# Python function 'date.fromordinal' to convert the serial date. 
# Ref 1: http://sociograph.blogspot.com/2011/04/how-to-avoid-gotcha-when-converting.html
# Ref 2: https://stackoverflow.com/questions/13965740/converting-matlabs-datenum-format-to-python

# 'photo_taken' is a list of years the photo was taken. In the dataset, it's assumed that the photo was taken 
# in the middle of the year

import time
from datetime import datetime, date, timedelta

def calculate_age(dob, taken_year):
# sanity check
#    print(taken_date.dtype)
#    print(dob.dtype)
# The following function takes only one value at a time, and becasue we're doing a subtraction to correct from ordinal
# date, need a max() to make sure there are no negatives
    birth_python = datetime.fromordinal(max(int(dob) - 366,1))   
    birth_year=birth_python.year
# today's date: to calculate current age, but doesn't make much sense 
#    current_year = date.today().year
    
    if (birth_python.month<=6):
        age_phototaken=taken_year-birth_year
#        age_current=current_year-birth_year
    else:
        age_phototaken=taken_year-birth_year-1
#        age_current=current_year-birth_year-1
#    return age_phototaken,age_current    
    return age_phototaken

age=complete_df.apply(lambda row: calculate_age(row['dob'],row['photo_taken']),axis=1)
complete_df['age']=age

print(complete_df.head(20))

#This works for one set of array values:
#age=[calculate_age(dob[i],photo_taken[i]) for i in range(len(dob))]

#if current age is needed:
#to unpack multiple values with list comprehension (returns tuple, so reverse by using map(list,..):
#age_phototaken,age_current =map(list,zip(*[calculate_age(dob[i],photo_taken[i]) for i in range(len(dob))]))


       dob  photo_taken                                       full_path  \
0   693726         1968   [01/nm0000001_rm124825600_1899-5-10_1968.jpg]   
1   693726         1970  [01/nm0000001_rm3343756032_1899-5-10_1970.jpg]   
2   693726         1968   [01/nm0000001_rm577153792_1899-5-10_1968.jpg]   
3   693726         1968   [01/nm0000001_rm946909184_1899-5-10_1968.jpg]   
4   693726         1968   [01/nm0000001_rm980463616_1899-5-10_1968.jpg]   
5   702986         1991  [02/nm0000002_rm1075631616_1924-9-16_1991.jpg]   
6   702986         2004  [02/nm0000002_rm1346607872_1924-9-16_2004.jpg]   
7   702986         2004  [02/nm0000002_rm1363385088_1924-9-16_2004.jpg]   
8   702986         1991  [02/nm0000002_rm1411175936_1924-9-16_1991.jpg]   
9   702986         2004  [02/nm0000002_rm1447271168_1924-9-16_2004.jpg]   
10  702986         2008  [02/nm0000002_rm1624085760_1924-9-16_2008.jpg]   
11  702986         1991  [02/nm0000002_rm1646056960_1924-9-16_1991.jpg]   
12  702986         1974  

In [59]:
complete_df.describe()

,dob,photo_taken,gender,face_score,second_face_score,age
count,523051.000000,523051.000000,511946.000000,5.230510e+05,217893.000000,523051.000000
mean,718712.859713,2004.663652,0.606074,-inf,2.444560,36.888909
std,13010.612963,11.358271,0.488619,NaN,1.064635,34.333007
min,47.000000,1940.000000,0.000000,-inf,0.730926,-74.000000
25%,715961.000000,2003.000000,0.000000,1.643745e+00,1.573953,27.000000
50%,719895.000000,2008.000000,1.000000,2.930596e+00,2.345420,35.000000
75%,723199.000000,2011.000000,1.000000,3.982567e+00,3.219398,44.000000
max,736011.000000,2015.000000,1.000000,7.381689e+00,6.395435,2014.000000


In [60]:
# Check validity of images
# Conditions:
# 1) gender: Ignore images with unknown gender (NaN if unknown)
# 2) face_score: Ignore images with no face (Inf implies that no face was found)
# 3) second_face_score: Images with second face will be ignored (NaN if no second face)
# 4) age: age range from -31->2014, only pick valid values
def get_valid(gender,face_score,second_face_score,age):
#    valid_images=True
    if np.isnan(gender):
        valid_images=False
    elif face_score<0.0 or np.isinf(face_score):
        valid_images=False
    elif ~np.isnan(second_face_score):
        valid_images=False
    elif age<0 or age>100:
        valid_images=False
    else:
        valid_images=True
    return valid_images
    
valid_images=complete_df.apply(lambda row: get_valid(row['gender'],row['face_score'],row['second_face_score'],row['age']) , axis=1)
complete_df['valid_images']=valid_images
print(complete_df['valid_images'].value_counts())
print(complete_df.head())

False    302176
True     220875
Name: valid_images, dtype: int64
      dob  photo_taken                                       full_path  \
0  693726         1968   [01/nm0000001_rm124825600_1899-5-10_1968.jpg]   
1  693726         1970  [01/nm0000001_rm3343756032_1899-5-10_1970.jpg]   
2  693726         1968   [01/nm0000001_rm577153792_1899-5-10_1968.jpg]   
3  693726         1968   [01/nm0000001_rm946909184_1899-5-10_1968.jpg]   
4  693726         1968   [01/nm0000001_rm980463616_1899-5-10_1968.jpg]   

   gender  face_score  second_face_score  age  valid_images  
0     1.0    1.459693           1.118973   69         False  
1     1.0    2.543198           1.852008   71         False  
2     1.0    3.455579           2.985660   69         False  
3     1.0    1.872117                NaN   69          True  
4     1.0    1.158766                NaN   69          True  


In [61]:
# Save valid images
valid_df=complete_df.loc[complete_df['valid_images'],['full_path','gender','age']].reset_index(drop=True)
print(valid_df.head())

                                        full_path  gender  age
0   [01/nm0000001_rm946909184_1899-5-10_1968.jpg]     1.0   69
1   [01/nm0000001_rm980463616_1899-5-10_1968.jpg]     1.0   69
2  [02/nm0000002_rm1346607872_1924-9-16_2004.jpg]     0.0   79
3  [02/nm0000002_rm1363385088_1924-9-16_2004.jpg]     0.0   79
4   [02/nm0000002_rm221957120_1924-9-16_1974.jpg]     0.0   49


In [62]:
valid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220875 entries, 0 to 220874
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   full_path  220875 non-null  object 
 1   gender     220875 non-null  float64
 2   age        220875 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 5.1+ MB


In [ ]:
from tqdm import tqdm
import cv2

#*** Set image resize dimensions here
img_dim=50

total_count=valid_df.full_path.count()
print(total_count)
#for i in tqdm(range(total_count)):

for i in range(total_count):
    valid_impath=parent_dir+str(full_path[i][0])
    img=cv2.imread(valid_impath)
#    img=cv2.resize(img,(img_dim,img_dim))



220875
